In [3]:
cd ..

/Users/wliao0504/Library/Mobile Documents/com~apple~CloudDocs/Repo/hca-lab/CLIF-MIMIC


/Users/wliao0504/.pyenv/versions/3.13.0/lib/python3.13/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
# src/tables/medication_admin_continuous.py
import numpy as np
import pandas as pd
import logging
from importlib import reload
import src.utils
reload(src.utils)
from src.utils import construct_mapper_dict, fetch_mimic_events, load_mapping_csv, \
    get_relevant_item_ids, find_duplicates, rename_and_reorder_cols, save_to_rclif, \
    convert_and_sort_datetime, setup_logging, search_mimic_items
from fuzzywuzzy import process

loaded configuration from config.json


In [ ]:
setup_logging()

MAC_COLUMNS = [
    "hospitalization_id", "med_order_id", "admin_dttm", "med_name", "med_category", "med_group", 
    "med_route_name", "med_route_category", "med_dose", "med_dose_unit", "mar_action_name", "mar_action_category"
]

MAC_COL_RENAME_MAPPER = {
    "dose": "med_dose",
    "rateuom": "med_dose_unit",
    "amountuom": "med_dose_unit",
    "new_mar": "mar_action_name", 
    "linkorderid": "med_order_id",
    "recorded_dttm": "admin_dttm"
}

MAC_MCIDE_URL = "https://raw.githubusercontent.com/clif-consortium/CLIF/main/mCIDE/clif_medication_admin_continuous_med_categories.csv"

def map_name_to_category(name, categories):
    '''
    Map a medication name to a category using fuzzy matching.
    '''
    match, score = process.extractOne(name, categories)
    return match if score >= 80 else None

def are_doses_close(doses):
    return (abs(doses.iloc[0] - doses.iloc[1]) / max(doses.iloc[0], doses.iloc[1])) <= 0.1

# drop the row with the shorter mar_action_name
def drop_shorter_action_name(group):
    if len(group) == 2 and are_doses_close(group['med_dose']):
        return group.loc[[group['mar_action_name'].str.len().idxmax()]]
    return group

In [8]:
mac_mapping = load_mapping_csv("mac")
mac_mapper = construct_mapper_dict(mac_mapping, "itemid", "med_category")

logging.info("parsing the mapping files to identify relevant items and fetch corresponding events...")
mac_item_ids = get_relevant_item_ids(
    mapping_df = mac_mapping, 
    decision_col = "decision", 
    excluded_labels = ["NO MAPPING", "UNSURE", "MAPPED ELSEWHERE", "NOT AVAILABLE", "TO MAP, ELSEWHERE"]
    ) 

mac_events = fetch_mimic_events(mac_item_ids)

In [14]:
mac_events.head()


,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,amount,amountuom,...,originalamount,originalrate,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,10000690,25860671,37081114,8787,2150-11-06 12:00:00,2150-11-06 12:01:00,2150-11-06 13:16:00,220949,100.000000,mL,...,100.000000,0.0,Dextrose 5%,Dextrose 5%,inputevents,Fluids/Intake,mL,Solution,NaN,NaN
1,10000690,25860671,37081114,8787,2150-11-06 13:00:00,2150-11-06 13:01:00,2150-11-06 13:16:00,220949,200.000000,mL,...,200.000000,0.0,Dextrose 5%,Dextrose 5%,inputevents,Fluids/Intake,mL,Solution,NaN,NaN
2,10000690,25860671,37081114,17393,2150-11-02 20:00:00,2150-11-02 22:45:00,2150-11-02 20:14:00,221749,5.475664,mg,...,60.000000,0.6,Phenylephrine,Phenylephrine,inputevents,Medications,mg,Solution,NaN,NaN
3,10000690,25860671,37081114,17393,2150-11-02 22:45:00,2150-11-02 23:36:00,2150-11-02 23:12:00,221749,1.410112,mg,...,54.524338,0.5,Phenylephrine,Phenylephrine,inputevents,Medications,mg,Solution,NaN,NaN
4,10000690,25860671,37081114,17393,2150-11-02 23:36:00,2150-11-03 00:35:00,2150-11-03 00:42:00,221749,1.305181,mg,...,53.114223,0.4,Phenylephrine,Phenylephrine,inputevents,Medications,mg,Solution,NaN,NaN


In [20]:
# s stands for simple    
mac_events_s = mac_events[[
    'subject_id', 'hadm_id', 'starttime',
    'endtime', 'storetime', 'statusdescription', 'itemid', 'amount', 'amountuom', 'rate',
    'rateuom', # 'orderid', 
    'linkorderid', # 'ordercategoryname',
    'totalamount', 'totalamountuom', 'originalamount', 'originalrate'
    ]].reset_index(drop = True)
# mac_events_s = convert_and_sort_datetime(mac_events_s)
# RESUME HERE

In [21]:
convert_and_sort_datetime(mac_events_s)

ValueError: time data "2189-06-27 07:40:00" doesn't match format "%Y-%b-%d %H:%M:%S", at position 18. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.